In [6]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


In [7]:
# Define image size and batch size
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Ensure the training dataset is correctly structured
train_dataset = train_datagen.flow_from_directory(
    'train/',  # Replace with your training data path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # Or 'binary' if it's binary classification
)

# Ensure the validation dataset is correctly structured
val_dataset = val_datagen.flow_from_directory(
    'valid/',  # Replace with your validation data path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # Or 'binary' if it's binary classification
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [8]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of VGG19
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_dataset.num_classes, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# # # Freeze the layers of VGG19
for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    verbose=1
)

Epoch 1/3


/Users/sammamsohail/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 403s 183ms/step - accuracy: 0.7048 - loss: 1.2060 - val_accuracy: 0.8785 - val_loss: 0.3594
Epoch 2/3
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 406s 185ms/step - accuracy: 0.8563 - loss: 0.4497 - val_accuracy: 0.9089 - val_loss: 0.2765
Epoch 3/3
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 404s 184ms/step - accuracy: 0.8794 - loss: 0.3674 - val_accuracy: 0.9132 - val_loss: 0.2580


In [9]:
model.save('model.h5')